In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("recommender").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/02 15:44:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [6]:
data = spark.read.csv("movielens_ratings.csv", inferSchema = True, header = True)

In [7]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [8]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [9]:
(training, test) = data.randomSplit([0.8, 0.2])

In [10]:
als = ALS(maxIter = 5, regParam = 0.01, userCol = "userId", itemCol = "movieId", ratingCol = "rating")

In [11]:
model = als.fit(training)

22/05/02 15:46:40 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/05/02 15:46:40 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/05/02 15:46:41 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [12]:
predictions = model.transform(test)

In [13]:
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      1|   1.0|     4|0.13013786|
|      1|   1.0|     6| 2.1072946|
|      1|   1.0|    14| 1.6732228|
|      1|   1.0|    28|-2.9360113|
|      6|   1.0|    12| 1.2895387|
|      6|   1.0|    17| 1.5895464|
|      3|   1.0|     7| 2.2554243|
|      3|   1.0|    13|  1.117585|
|      5|   1.0|     9| 1.4005597|
|      5|   1.0|    14|0.75990117|
|      9|   1.0|     6| 2.4114118|
|      9|   1.0|    14|0.20732616|
|      9|   1.0|    16| 3.2930079|
|      4|   1.0|    23|0.50531626|
|      4|   4.0|    26| 0.9151591|
|      7|   3.0|     3| 1.4156462|
|      7|   2.0|    29|0.42501086|
|      2|   2.0|     7| 1.2387019|
|      2|   4.0|     8|0.06599331|
|      2|   1.0|    19| 0.7757074|
+-------+------+------+----------+
only showing top 20 rows



In [14]:
evaluator = RegressionEvaluator(metricName = "rmse", labelCol = "rating", predictionCol = "prediction") 

In [15]:
rmse = evaluator.evaluate(predictions)

In [16]:
print("RMSE")
print(rmse)

RMSE
1.9218435441235755


In [17]:
single_user = test.filter(test["userId"] == 11).select(["movieId", "userId"])

In [18]:
single_user.show()


+-------+------+
|movieId|userId|
+-------+------+
|     11|    11|
|     12|    11|
|     18|    11|
|     19|    11|
|     20|    11|
|     23|    11|
|     38|    11|
|     40|    11|
|     43|    11|
|     47|    11|
|     67|    11|
|     75|    11|
|     78|    11|
|     89|    11|
|     90|    11|
+-------+------+



In [19]:
recommendations = model.transform(single_user)


In [20]:
recommendations.orderBy("prediction", ascending=False).show()


+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     40|    11|  6.3951516|
|     12|    11|   5.218531|
|     75|    11|  4.8531785|
|     18|    11|   4.804379|
|     20|    11|  3.5914714|
|     47|    11|  3.5575843|
|     67|    11|  3.2241821|
|     90|    11|  2.8318176|
|     23|    11|  2.8161323|
|     43|    11|   2.701276|
|     89|    11|  2.1862082|
|     78|    11|  2.0673306|
|     38|    11|  1.1342851|
|     11|    11|  0.6084006|
|     19|    11|-0.17527628|
+-------+------+-----------+

